# Inseq：可视化解释LLM的输出

> [HW7: Understand what Generative AI is thinking](https://colab.research.google.com/drive/1Xnz0GHC0yWO2Do0aAYBCq9zL45lbiRjM?usp=sharing#scrollTo=UFOUfh2k1jFN) 中文镜像版
>
> 指导文章：[12. Inseq 特征归因：可视化解释 LLM 的输出](https://github.com/Hoper-J/LLM-Guide-and-Demos-zh_CN/blob/master/Guide/12.%20Inseq%20特征归因：可视化解释%20LLM%20的输出.md)

Feature attribution 特征归因，你可以将其当做对输出的解释，就像图像分类模型可以可视化关注区域一样，LLM 一样也可以。

In [1]:
!pip install inseq==0.5.0
!pip install transformers==4.40.2
!pip install bitsandbytes==0.43.1
!pip install -U accelerate==0.28.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.7/136.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 85.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## 可视化翻译任务

In [4]:
import inseq

# 要使用的归因方法列表
attribution_methods = ['saliency', 'attention']

for method in attribution_methods:
    print(f"======= 归因方法: {method} =======")
    # 加载中译英模型并设置归因方法
    model = inseq.load_model("Helsinki-NLP/opus-mt-zh-en", method)
#     model = inseq.load_model("opus-mt-zh-en", method)  # 本地

    # 使用指定的归因方法对输入文本进行归因
    attribution_result = model.attribute(
        input_texts="今天学到了很多，我很开心。",
        step_scores=["probability"],
    )

    # 从tokenizer中去除 '▁' 前缀以避免混淆（可以忽略这段代码）
    for attr in attribution_result.sequence_attributions:
        for item in attr.source:
            item.token = item.token.replace('▁', '')
        for item in attr.target:
            item.token = item.token.replace('▁', '')

    # 显示归因结果
    attribution_result.show()

======= 归因方法: saliency =======


Attributing with saliency...: 100%|██████████| 16/16 [00:05<00:00,  2.54it/s]


,I,',ve,learned,a,lot,today,",",and,I,',m,happy,.,</s>
今天,0.117,0.138,0.102,0.119,0.107,0.114,0.297,0.088,0.105,0.075,0.078,0.077,0.059,0.093,0.127
学到,0.274,0.244,0.231,0.397,0.272,0.269,0.206,0.172,0.168,0.182,0.143,0.159,0.144,0.193,0.208
了很多,0.144,0.101,0.115,0.113,0.24,0.273,0.116,0.118,0.105,0.075,0.07,0.074,0.071,0.102,0.096
",",0.061,0.048,0.055,0.031,0.036,0.045,0.047,0.105,0.071,0.039,0.034,0.03,0.029,0.04,0.063
我,0.08,0.07,0.077,0.045,0.044,0.042,0.044,0.112,0.094,0.099,0.084,0.065,0.07,0.072,0.062
很开心,0.174,0.21,0.266,0.171,0.147,0.133,0.137,0.222,0.246,0.372,0.427,0.44,0.473,0.32,0.188
。,0.092,0.112,0.092,0.066,0.09,0.072,0.094,0.111,0.129,0.096,0.096,0.093,0.093,0.111,0.167
</s>,0.059,0.076,0.063,0.058,0.064,0.054,0.058,0.073,0.083,0.062,0.069,0.064,0.061,0.069,0.091
probability,0.44,0.396,0.267,0.725,0.65,0.826,0.722,0.342,0.551,0.817,0.762,0.888,0.491,0.788,0.895


======= 归因方法: attention =======


Attributing with attention...: 100%|██████████| 16/16 [00:01<00:00, 12.40it/s]


,I,',ve,learned,a,lot,today,",",and,I,',m,happy,.,</s>
今天,0.09,0.073,0.054,0.071,0.057,0.055,0.191,0.16,0.038,0.023,0.023,0.018,0.02,0.015,0.017
学到,0.184,0.211,0.085,0.245,0.217,0.055,0.063,0.033,0.028,0.023,0.03,0.019,0.023,0.02,0.013
了很多,0.12,0.092,0.073,0.117,0.257,0.343,0.205,0.023,0.029,0.021,0.016,0.016,0.021,0.015,0.014
",",0.08,0.046,0.031,0.022,0.022,0.024,0.045,0.062,0.133,0.079,0.022,0.015,0.013,0.009,0.055
我,0.053,0.145,0.061,0.039,0.017,0.018,0.024,0.098,0.123,0.133,0.184,0.082,0.115,0.025,0.021
很开心,0.061,0.18,0.123,0.138,0.063,0.044,0.061,0.085,0.168,0.23,0.356,0.236,0.461,0.248,0.037
。,0.088,0.048,0.059,0.036,0.028,0.045,0.032,0.071,0.115,0.117,0.052,0.054,0.04,0.092,0.279
</s>,0.324,0.205,0.515,0.332,0.34,0.415,0.379,0.469,0.367,0.374,0.317,0.561,0.307,0.576,0.565
probability,0.44,0.396,0.267,0.725,0.65,0.826,0.722,0.342,0.551,0.817,0.762,0.888,0.491,0.788,0.895


## 可视化文本生成任务

### 下载 GPT-2 XL 模型

如果只是为了查看的话，可以取消注释部分下载 GPT-2，因为 GPT-2 XL 下载完需要占用 30G，记得注释原来的部分。

In [3]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig


# 创建一个 BitsAndBytesConfig 对象，用于配置量化选项
bnb_config = BitsAndBytesConfig(load_in_8bit=True)
model = AutoModelForCausalLM.from_pretrained(
    "gpt2-xl",  # "gpt2"
    quantization_config=bnb_config,
    device_map={"": 0}
)

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`

In [ ]:
import inseq


# 定义要使用的归因方法列表
attribution_methods = ['saliency', 'attention']

for method in attribution_methods:
    print(f"======= 归因方法: {method} =======")
    # 使用指定的归因方法加载模型
    inseq_model = inseq.load_model(model, method)

    # 对输入文本进行归因分析
    attribution_result = inseq_model.attribute(
        input_texts="Hello world",
        step_scores=["probability"],

    )

    # 清理 tokenizer 中的特殊字符（可选）
    for attr in attribution_result.sequence_attributions:
        for item in attr.source:
            item.token = item.token.replace('Ġ', '')
        for item in attr.target:
            item.token = item.token.replace('Ġ', '')

    # 显示归因结果
    # 注意是一个个生成的  所以相关性是阶梯状的
    attribution_result.show()

======= 归因方法: saliency =======


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Attributing with saliency...: 100%|██████████| 20/20 [00:11<00:00,  1.54it/s]


,"""",Ċ,Ċ,The,first,thing,you,'ll,notice,is,that,the,code,is,very,simple,.,It
Hello,0.647,0.505,0.354,0.401,0.392,0.343,0.353,0.263,0.267,0.302,0.304,0.237,0.365,0.22,0.25,0.255,0.146,0.175
world,0.353,0.315,0.22,0.237,0.254,0.199,0.242,0.179,0.173,0.212,0.197,0.164,0.245,0.148,0.144,0.162,0.084,0.112
"""",,0.18,0.186,0.159,0.127,0.129,0.102,0.091,0.085,0.094,0.073,0.072,0.045,0.057,0.047,0.056,0.058,0.06
Ċ,,,0.241,0.092,0.055,0.056,0.044,0.045,0.043,0.042,0.039,0.065,0.036,0.042,0.033,0.032,0.035,0.029
Ċ,,,,0.11,0.057,0.062,0.045,0.052,0.05,0.041,0.035,0.039,0.026,0.037,0.031,0.028,0.037,0.026
The,,,,,0.114,0.073,0.066,0.058,0.044,0.04,0.045,0.04,0.02,0.041,0.025,0.029,0.03,0.035
first,,,,,,0.138,0.048,0.074,0.058,0.046,0.046,0.039,0.022,0.051,0.03,0.029,0.035,0.031
thing,,,,,,,0.101,0.101,0.084,0.049,0.049,0.053,0.028,0.043,0.037,0.033,0.044,0.037
you,,,,,,,,0.137,0.079,0.047,0.048,0.049,0.028,0.043,0.036,0.051,0.044,0.052
'll,,,,,,,,,0.115,0.048,0.051,0.063,0.032,0.052,0.053,0.038,0.053,0.054


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


======= 归因方法: attention =======


Attributing with attention...: 100%|██████████| 20/20 [00:04<00:00,  4.26it/s]


,"""",Ċ,Ċ,The,first,thing,you,'ll,notice,is,that,the,code,is,very,simple,.,It
Hello,0.867,0.758,0.674,0.619,0.619,0.636,0.612,0.626,0.623,0.584,0.518,0.528,0.552,0.572,0.555,0.568,0.496,0.446
world,0.133,0.109,0.054,0.068,0.055,0.038,0.041,0.03,0.024,0.028,0.036,0.04,0.034,0.024,0.015,0.014,0.017,0.019
"""",,0.133,0.086,0.084,0.061,0.048,0.043,0.037,0.031,0.03,0.035,0.038,0.035,0.025,0.019,0.018,0.019,0.02
Ċ,,,0.186,0.114,0.091,0.075,0.068,0.061,0.054,0.059,0.058,0.052,0.046,0.042,0.038,0.036,0.046,0.043
Ċ,,,,0.116,0.093,0.072,0.064,0.049,0.041,0.044,0.056,0.055,0.04,0.03,0.024,0.022,0.026,0.034
The,,,,,0.08,0.078,0.061,0.041,0.035,0.033,0.043,0.036,0.031,0.025,0.019,0.018,0.02,0.022
first,,,,,,0.054,0.059,0.037,0.031,0.032,0.026,0.022,0.02,0.014,0.013,0.012,0.013,0.014
thing,,,,,,,0.052,0.058,0.044,0.042,0.026,0.02,0.017,0.014,0.013,0.011,0.011,0.012
you,,,,,,,,0.062,0.06,0.04,0.03,0.022,0.018,0.018,0.013,0.011,0.015,0.014
'll,,,,,,,,,0.056,0.048,0.027,0.02,0.015,0.016,0.014,0.012,0.013,0.013
